# Import Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
from sklearn import tree
from hijridate import Hijri, Gregorian
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf

# Data Selection

In [ ]:
raw_datasets: dict[str, pd.DataFrame] = {}

# Specify where datasets stored
raw_dataset_dir = Path("../dataset/raw")
paths = sorted(raw_dataset_dir.glob("*.xlsx"))

# Check if datasets exists
if not paths:
    raise FileNotFoundError(
        f"Tidak ditemukan dataset .xlsx pada {raw_dataset_dir.resolve()}"
    )

# Print loaded dataset
print(f"Loaded Dataset :")
for path in paths:
    name = path.stem
    print(f"{path.name}")
    # Read dataset, Drop unnecessary column, and Revise columns name
    df_raw = (
        pd.read_excel(path)
        .drop(columns=["No"])
        .rename(columns={"Komoditas (Rp)": "Province"})
    )
    # Drop summary row
    df_raw = df_raw[df_raw["Province"] != "Semua Provinsi"].reset_index(drop=True)
    raw_datasets[name] = df_raw.reset_index(drop=True)
    # Print dataframe
    print(df_raw.head(5))

# Data Cleaning

In [ ]:
# Specify where cleaned dataset stored
clean_dataset_dir = Path("../dataset/clean")
clean_dataset_dir.mkdir(parents=True, exist_ok=True)

cleaned_datasets: dict[str, pd.DataFrame] = {}

# Loop process to clean each raw dataset
for name, df in raw_datasets.items():
    # Select Date
    date_cols = [c for c in df.columns if c not in ["Province"]]

    # Change dataset format from wide to long
    df_long = df.melt(
        id_vars=["Province"],
        value_vars=date_cols,
        var_name="Date",
        value_name="Price",
    )

    # Clean date column by removing space and change date format
    df_long["Date"] = pd.to_datetime(
        df_long["Date"].str.replace(" ", "").str.strip(),
        format="%d/%m/%Y",
        errors="coerce",
    )

    # Clean price column by replacing "-" with NaN, remove comma, and format to numeric
    df_long["Price"] = (
        df_long["Price"]
        .astype(str)
        .str.strip()
        .replace("-", np.nan)
        .str.replace(",", "", regex=False)
    )
    df_long["Price"] = pd.to_numeric(df_long["Price"], errors="coerce")

    # Forward/Backward Fill Based Each Province
    df_long = (
        df_long.groupby("Province", sort=False, group_keys=False)[["Date", "Price", "Province"]]
        .apply(
            lambda g: (
                g.set_index("Date")
                .reindex(
                    pd.date_range(
                        start=g["Date"].min(),
                        end=g["Date"].max(),
                        freq="D",
                    )
                )
                .assign(Province=lambda x: x["Province"].ffill().bfill())
                .assign(
                    Price=lambda x: x["Price"].ffill().bfill()
                )
                .rename_axis("Date")
                .reset_index()
            )
        )
        .reset_index(drop=True)
    )
    
    # Sort by date and province
    df_long = df_long.sort_values(["Date", "Province"]).reset_index(drop=True)

    # Round price
    df_long["Price"] = df_long["Price"].round().astype(int)

    # Export cleaned dataframe
    file_name = f"Clean_{name}.xlsx"
    df_long.to_excel(clean_dataset_dir / file_name, index=False)

    # Print preview cleaned dataframe
    print(df_long.head(5))
    print("...")
    print(df_long.tail(5))
    print("\n")

    # Save to dictionary
    cleaned_datasets[name] = df_long


# Data Integration

In [ ]:
merged_dataset_path = Path("../dataset/merged/merged_datasets.xlsx")
merged_dataset_path.parent.mkdir(parents=True, exist_ok=True)

frames: list[pd.DataFrame] = []

for name, df_transform in cleaned_datasets.items():
    new_df = df_transform.copy()
    frames.append(new_df)

# Merge all dataset
df_merged = pd.concat(frames, ignore_index=True)

# Sort and drop duplicate each province and date
# If there's duplicate data, get the last one
df_merged = df_merged.drop_duplicates(
    subset=["Province", "Date"], keep="last"
).reset_index(drop=True)

# Export merged dataframe
df_merged.to_excel(merged_dataset_path)

# Print preview merged dataframe
print(df_merged.head(5))
print("...")
print(df_merged.tail(5))

# Data Transformation

In [ ]:
# Check if date is eid
def eid_delta_days(ts: pd.Timestamp) -> int:
    g = Gregorian(ts.year, ts.month, ts.day)
    h = g.to_hijri()

    # 1 Syawal = 10 Hijriah
    eid_h = Hijri(h.year, 10, 1)
    eid_g = eid_h.to_gregorian()

    eid_date = pd.Timestamp(eid_g.year, eid_g.month, eid_g.day)
    return (ts.normalize() - eid_date).days

# Flag date based on eid 14 day window
def eid_flags(ts: pd.Timestamp):
    d = eid_delta_days(ts)
    before = 1 if -7 <= d <= -1 else 0
    day     = 1 if d == 0 else 0
    after  = 1 if 1 <= d <= 6 else 0
    return pd.Series([before, day, after],
                     index=["before_eid", "eid", "after_eid"])

# Specify where transformed dataset stored
transformed_dataset_path = Path("../dataset/transformed/transformed_datasets.xlsx")
transformed_dataset_path.parent.mkdir(parents=True, exist_ok=True)

df = df_merged.copy()

# Data type normalization
df["Date"] = pd.to_datetime(df["Date"])
df["Province"] = df["Province"].astype("string")
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

# Feature Engineering
## 1) Province → numeric (ID)
# Calculate average price for each province
province_mean_prices = df.groupby("Province")["Price"].mean().sort_values()
# Create a mapping dictionary from province name to a rank (0 to 33)
province_mapping = {province: i for i, province in enumerate(province_mean_prices.index)}
df["Province_id"] = df["Province"].map(province_mapping)

print("Province Mapping (Lowest to Highest Price):")
print(province_mapping)

## 2) Lag features for each province
for lag in [1, 14]:
    df[f"lag_{lag}"] = df.groupby("Province", group_keys=False)["Price"].shift(lag)

## 3) Moving holiday features
df[["before_eid", "eid", "after_eid"]] = df["Date"].apply(eid_flags)

## 4) Time based features
df["month"] = df["Date"].dt.month
df["year"] = df["Date"].dt.year

# Drop row data with no lag features
df_transform = df.dropna(subset=["lag_1", "lag_14"]).reset_index(drop=True)

# Export transformed dataframe
df_transform.to_excel(transformed_dataset_path)

# Print preview transformed dataframe
print("\n")
print(df_transform.head(5))
print("...")
print(df_transform.tail(5))
print("\n")

# Data Mining

In [ ]:
# Specify where mined dataset stored
mined_dataset_path = Path("../dataset/mined/rfr_datasets.xlsx")
mined_dataset_path.parent.mkdir(parents=True, exist_ok=True)

df_mining = df_transform.copy()

# Configure Parameters, Train Size, etc
RFR_PARAMS = dict(
    n_estimators=60,
    max_depth=None,
    min_samples_split=20,
    min_samples_leaf=5,
    max_features="sqrt",
    bootstrap=True,
    random_state=42,
    n_jobs=-1,
)

# Amount of data used for training
train_size = 0.9

# Specify Feature Columns and Target Column
FEATURE_COLS = ["Province_id", "lag_1", "lag_14", "before_eid", "eid", "after_eid", "month", "year"]
TARGET_COL = "Price"

## Creating Model for Random Forest Regression

In [ ]:
# Specify X for features and y for target
X = df_mining[FEATURE_COLS]
y = df_mining[TARGET_COL]

# Time-based train-test split
split_index = int(len(df_mining) * train_size)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

model = RandomForestRegressor(**RFR_PARAMS)

# Train model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Model Evaluation

In [ ]:
rmse = root_mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

# Knowledge Presentation

In [ ]:
print("\n")
print("=== Random Forest Regression ===")
print(f"Train Data : {train_size*100}%")
print(f"RMSE : {rmse:,.2f}")
print(f"MAPE : {mape:.2f}%")

# Save prediction results for analysis
mined_df = pd.DataFrame(
    {
        "Date": df_mining.iloc[split_index:]["Date"].values,
        "Province": df_mining.iloc[split_index:]["Province"].values,
        "Actual": y_test.values,
        "Prediction": y_pred,
    }
)

# Export dataframe
mined_df.to_excel(mined_dataset_path, index=False)

# Show scatter plot between actual data vs prediction
plt.figure(figsize=(5, 5))
plt.scatter(mined_df["Actual"], mined_df["Prediction"], alpha=0.6)
max_val = max(mined_df["Actual"].max(), mined_df["Prediction"].max())
min_val = min(mined_df["Actual"].min(), mined_df["Prediction"].min())
plt.plot([min_val, max_val], [min_val, max_val], "r--", linewidth=2)

plt.title("Actual vs Prediction")
plt.xlabel("Actual")
plt.ylabel("Prediction")
plt.tight_layout()
plt.show()

# Show tree visualization
n_trees_to_view = 3

for i in range(n_trees_to_view):
    plt.figure(figsize=(20, 10))
    tree.plot_tree(model.estimators_[i], 
                   feature_names=X.columns, 
                   filled=True, 
                   max_depth=2,
                   fontsize=12)
    plt.title(f"Visualisasi Estimator (Pohon) ke-{i+1} dalam Random Forest")
    plt.show()

# Optional

## Generate Sugar Price History Plot

In [ ]:
df = df_merged.copy()

# Specify where output will be stored
combined_path = Path("../output/plots/sugar_price_history.png")
province_dir = Path("../output/plots/province_price_history/")
combined_path.parent.mkdir(parents=True, exist_ok=True)
province_dir.mkdir(parents=True, exist_ok=True)

# All provinces combined plot
plt.figure(figsize=(12, 6))

for prov, g in df.groupby("Province"):
    plt.plot(g["Date"], g["Price"], label=prov, linewidth=0.8, alpha=0.7)

plt.title("Riwayat Harga Gula per Provinsi")
plt.xlabel("Tanggal")
plt.ylabel("Harga (Rp)")
plt.legend(
    title="Provinsi", fontsize=6, ncol=2, bbox_to_anchor=(1.05, 1), loc="upper left"
)
plt.tight_layout()
plt.savefig(combined_path)
plt.close()

# Each individual province plot
for prov, g in df.groupby("Province"):
    plt.figure(figsize=(10, 5))
    plt.plot(g["Date"], g["Price"], linewidth=1.5)

    plt.title(f"Riwayat Harga Gula - {prov}")
    plt.xlabel("Tanggal")
    plt.ylabel("Harga (Rp)")
    plt.tight_layout()

    file_name = prov.replace(" ", "_").replace("/", "_") + ".png"
    plt.savefig(province_dir / file_name)
    plt.close()

## Generate PACF Plot

In [ ]:
df = df_transform.copy()
y_full = df['Price'].astype(float)
max_lag = 31

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 4))
plot_pacf(y_full, ax=ax, lags=max_lag, method="ywm")    # Method Yule-Walker Modified
ax.set_title(f"PACF Harga Gula Pasir (1 s.d. {max_lag} lag)")
ax.set_xlabel("Lag")
ax.set_ylabel("Partial Autocorrelation")
plt.tight_layout()
plt.show()

# Create PACF value as numeric + confidence interval
pacf_vals, confint = pacf(
    y_full,
    nlags=max_lag,
    alpha=0.05, # 95% confidence interval
    method="ywm"
)

# Find significant lags
significant_lags = [
    lag for lag in range(1, len(pacf_vals))
    if (confint[lag, 0] > 0) or (confint[lag, 1] < 0)
]

print("Lag yang signifikan menurut PACF:", significant_lags)

## Compare Between Actual and Prediction Data

In [ ]:
# Defined target range
horizon = 180
max_date = df_mining["Date"].max()
range_target = max_date - pd.Timedelta(days=horizon)

# Specify where outputs will be stored
output_dir = Path("../output/xlsx")
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / f"rfr_{horizon}days_comparison.xlsx"
compare_plot_dir = Path("../output/plots/compare")
compare_plot_dir.mkdir(parents=True, exist_ok=True)

# Split train / test by year
train_mask = df_mining["Date"] < range_target
test_mask = df_mining["Date"] >= range_target

# Check if data available for train and test
if not train_mask.any():
    raise ValueError(f"Tidak ada data {horizon} hari untuk training.")
if not test_mask.any():
    raise ValueError(f"Tidak ada data {horizon} hari untuk pengujian.")

# Sort dataframe by Date
df_train = df_mining[train_mask].sort_values(["Date", "Province"])
df_test = df_mining[test_mask].sort_values(["Date", "Province"])

# Create X, y train and test
X_train = df_train[FEATURE_COLS]
y_train = df_train[TARGET_COL]

X_test = df_test[FEATURE_COLS]
y_test = df_test[TARGET_COL]

# Create Random Forest Regression Model
model = RandomForestRegressor(**RFR_PARAMS)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate Model
rmse = root_mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\n")
print(f"=== Comparison for {horizon} days ===")
print(f"Train data up to: {df_train['Date'].max().date()}")
print(f"RMSE : {rmse:,.2f}")
print(f"MAPE : {mape:.2f}%")

# Export result
result_df = pd.DataFrame(
    {
        "Date": df_test["Date"].values,
        "Province": df_test["Province"].values,
        "Actual": y_test.values,
        "Prediction": y_pred,
    }
)

result_df.to_excel(output_path, index=False)

# Plot comparison result for each province
for prov in sorted(result_df["Province"].unique()):
    sub = result_df[result_df["Province"] == prov][["Date", "Actual", "Prediction"]]

    fig, ax = plt.subplots(figsize=(12, 5))

    ax.plot(sub["Date"], sub["Actual"], label="Actual", linewidth=1.2)
    ax.plot(sub["Date"], sub["Prediction"], label="Prediction", linewidth=1.2)

    ax.set_title(f"Riwayat Harga Aktual vs Prediksi ({prov} - {horizon} hari)")
    ax.set_xlabel("Tanggal")
    ax.set_ylabel("Harga (Rp)")
    ax.legend()

    fig.tight_layout()
    safe_filename = prov.replace("/", "_").replace(" ", "_")
    fig.savefig(compare_plot_dir / f"{safe_filename}.png")

    plt.close(fig)

## Forecast Future Data

In [ ]:
df = df_transform.copy()

X = df[FEATURE_COLS]
y = df[TARGET_COL]

# Create Random Forest Regression Model
model = RandomForestRegressor(**RFR_PARAMS)
model.fit(X_train, y_train)

# Specify how much data should be predicted
horizon = 180

forecast_results = []
provinces = df["Province"].unique()

# Specify where forecast output will be stored
forecast_plot_dir = Path("../output/plots/forecast/")
forecast_plot_dir.mkdir(parents=True, exist_ok=True)
forecast_output_path = Path("../output/xlsx/rfr_forecast_future_data.xlsx")
forecast_output_path.parent.mkdir(parents=True, exist_ok=True)

# Forecast loop each province
for prov in provinces:
    g = df[df["Province"] == prov].copy()
    g = g.sort_values("Date").reset_index(drop=True)

    last_date = g["Date"].max()
    last_rows = g.tail(30).copy()

    future_rows = []

    prov_id = g.iloc[-1]["Province_id"]

    # Forecast loop until specified horizon
    for i in range(1, horizon + 1):
        next_date = last_date + pd.Timedelta(days=i)
        
        # Time Based Features
        month = next_date.month
        year  = next_date.year

        # Lag Features
        lag_1  = last_rows.iloc[-1]["Price"]
        lag_14  = last_rows.iloc[-14]["Price"]  if len(last_rows) >= 14  else lag_1

        # Eid Features
        gdate = Gregorian(next_date.year, next_date.month, next_date.day)
        h = gdate.to_hijri()

        # 1 Syawal (Idul Fitri)
        eid_h = Hijri(h.year, 10, 1)
        eid_g = eid_h.to_gregorian()
        eid_date = pd.Timestamp(eid_g.year, eid_g.month, eid_g.day)

        delta = (next_date.normalize() - eid_date).days

        before_eid = 1 if -7 <= delta <= -1 else 0
        eid        = 1 if delta == 0 else 0
        after_eid  = 1 if 1 <= delta <= 6 else 0

        # Arrange future data features
        X_future = pd.DataFrame({
            "Province_id": [prov_id],
            "lag_1": [lag_1],
            "lag_14": [lag_14],
            "before_eid": [before_eid],
            "eid": [eid],
            "after_eid": [after_eid],
            "month": [month],
            "year": [year]
        })

        predicted_price = model.predict(X_future)[0]

        forecast_results.append({
            "Date": next_date,
            "Province": prov,
            "Prediction": round(predicted_price),
        })

        future_rows.append({
            "Date": next_date,
            "Price": predicted_price
        })

        # Update series
        last_rows = pd.concat([
            last_rows,
            pd.DataFrame([{
                "Date": next_date,
                "Price": predicted_price,
                "Province": prov,
                "Province_id": prov_id,
            }])
        ], ignore_index=True)


    # Plot
    future_df = pd.DataFrame(future_rows)

    plt.figure(figsize=(12, 5))
    plt.plot(g["Date"], g["Price"], label="Historical", linewidth=1.5)
    plt.plot(future_df["Date"], future_df["Price"], label=f"Forecast ({horizon} days)", linewidth=1.5)

    plt.title(f"Historical + Forecast Harga Gula - {prov}")
    plt.xlabel("Tanggal")
    plt.ylabel("Harga (Rp)")
    plt.legend()
    plt.tight_layout()

    file_name = prov.replace(" ", "_").replace("/", "_") + "_forecast.png"
    plt.savefig(forecast_plot_dir / file_name, dpi=150, bbox_inches="tight")
    plt.close()

# Export Forecast Dataframe
df_forecast = pd.DataFrame(forecast_results)
df_forecast = df_forecast.sort_values(["Province", "Date"]).reset_index(drop=True)
df_forecast.to_excel(forecast_output_path, index=False)